In [3]:
import sys
import os

import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import plotly.graph_objects as go # Nick did this, idk if it's standard but it's cute cause then you get go.Figure()
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [4]:
def get_file_data(file_paths) -> pd.DataFrame:
    metric_data = pd.DataFrame()

    for path in file_paths:
        # Import the data
        metric_datum = pd.read_csv(path, skiprows=1, sep='\t', low_memory=False)

        # Rename columns to have value and units, and remove the previous row for units
        metric_datum.rename(columns={col: f"{col} {metric_datum[col][0]}" for col in metric_datum.columns}, inplace=True)
        metric_datum = metric_datum.drop(0)

        # Convert all values to floats
        metric_datum = metric_datum.apply(pd.to_numeric, errors='coerce')

        # Drop any rows with NaN values (if necessary)
        metric_datum = metric_datum.dropna()

        # Reset index
        metric_datum.reset_index(drop=True, inplace=True)

        metric_data = pd.concat([metric_data, metric_datum])
        print(len(metric_datum))

    print(f"Loaded {len(metric_data)} columns.")

    return metric_data

def get_run_data(selected_runs) -> pd.DataFrame:
    file_names = [f'./RunData_Cornering_ASCII_SI_Round9/B2356run{rn}.dat' for rn in selected_runs]
    return get_file_data(file_names)

def get_all_raw_data() -> pd.DataFrame:
    data_files = []
    
    for root, _, files in os.walk("RawData_Cornering_ASCII_SI_Round9_Runs1to15 (1)"):
        for filename in files:
            data_files.append(os.path.join(root, filename))
    
    return get_file_data(data_files)


In [34]:
SELECTED_RUNS = [4, 5, 6, 8, 9]
SELECTED_RUNS = [2] + list(range(4, 26)) + list(range(27, 45)) + [46, 49]

# all_data = get_run_data(SELECTED_RUNS)
all_data = get_all_raw_data()


79851
210528
220043
79849
210550
220034
79861
127192
101287
220017
79867
210565
220001
Loaded 2059645 columns.


In [6]:
print(all_data["FY N"].describe())
print()
print(all_data["IA deg"].describe())

count    2.059645e+06
mean     8.139696e+01
std      9.162089e+02
min     -3.046500e+03
25%     -4.610000e+01
50%      1.610000e+00
75%      2.722300e+02
max      3.001880e+03
Name: FY N, dtype: float64

count    2.059645e+06
mean     1.225114e+00
std      1.741185e+00
min     -5.017000e+00
25%      4.000000e-03
50%      1.300000e-02
75%      2.005000e+00
max      4.031000e+00
Name: IA deg, dtype: float64


In [33]:
from scipy.interpolate import LinearNDInterpolator

points = all_data[["FZ N", "IA deg"]]
values = all_data["FY N"]

interp = LinearNDInterpolator(points, values)

print(interp)

In [ ]:
CA_YELLOW = "#FDB515"
BK_BLUE = "#002676"

LABEL_COLOR = BK_BLUE
TICK_COLOR = CA_YELLOW

SCENE_BACKGROUND = "white" #"Greenscreen", to photoshop out background
PLANE_COLOR = "#242424"

Fz_domain = np.linspace(all_data["FZ N"].min(), all_data["FZ N"].max(), 100, endpoint=True)[1:-1]
IA_domain = np.linspace(all_data["IA deg"].min(), all_data["IA deg"].max(), 100, endpoint=True)[1:-1]

Fz_grid, IA_grid = np.meshgrid(Fz_domain, IA_domain)

print(Fz_grid, IA_grid)

Fy_values = interp(Fz_grid, IA_grid)

# fig = go.Figure(data=[go.Surface(
#         x=Fz_domain,
#         y=IA_domain,
#         z=Fy_values
#     )])

# all_data = all_data.sample(10000)
reduced_data = all_data[all_data["FY N"] > 0].sample(50000)

fig = go.Figure(data = [
    go.Scatter3d(
        x=reduced_data["FZ N"],
        y=reduced_data["IA deg"],
        z=reduced_data["FY N"],
        mode='markers'
    )
])

fig.update_layout(
    scene=dict(
        bgcolor=SCENE_BACKGROUND,
        xaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Load Force (N)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR, size=16),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
        yaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Camber (deg)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR, size=16),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
        zaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Lateral Force (N)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR, size=16),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
    ),
    title=dict(
        x=0.5,
        xanchor='center',
        text='Lateral Force vs Load Force and Camber',
        font=dict(
            size=36,
        )
    ),
)

fig.write_html("camber_out/figures/3d_plot.html")

[[-1717.27363636 -1699.01727273 -1680.76090909 ...    17.08090909
     35.33727273    53.59363636]
 [-1717.27363636 -1699.01727273 -1680.76090909 ...    17.08090909
     35.33727273    53.59363636]
 [-1717.27363636 -1699.01727273 -1680.76090909 ...    17.08090909
     35.33727273    53.59363636]
 ...
 [-1717.27363636 -1699.01727273 -1680.76090909 ...    17.08090909
     35.33727273    53.59363636]
 [-1717.27363636 -1699.01727273 -1680.76090909 ...    17.08090909
     35.33727273    53.59363636]
 [-1717.27363636 -1699.01727273 -1680.76090909 ...    17.08090909
     35.33727273    53.59363636]] [[-4.92560606 -4.92560606 -4.92560606 ... -4.92560606 -4.92560606
  -4.92560606]
 [-4.83421212 -4.83421212 -4.83421212 ... -4.83421212 -4.83421212
  -4.83421212]
 [-4.74281818 -4.74281818 -4.74281818 ... -4.74281818 -4.74281818
  -4.74281818]
 ...
 [ 3.75681818  3.75681818  3.75681818 ...  3.75681818  3.75681818
   3.75681818]
 [ 3.84821212  3.84821212  3.84821212 ...  3.84821212  3.84821212
   3.

In [6]:
# import plotly.express as px

# fig = px.scatter(all_data, x="IA deg", y="FY N", 
#                  labels=dict(x="Camber (deg)", y="Lateral Force (N)"),
#                  title="Lateral Force over Camber (All Data)"
# )

# fig.update_layout(
#     width=800,  # Set the width in pixels
#     height=600, # Set the height in pixels
# )